# Formulas used in Fractalaxy

 [Fractalaxy](https://github.com/bfxdev/fractalaxy) relies on vector and complex number transformations. This Jupyter notebook shows the underlying concepts and explains where the formulas come from.

## Map Controller 2D

`MapController2D` is a GDScript attached to the main rendering node. It receives `InputEvent` signals in its `_input` function and maintain values of exported variables used by the renderer. This script is generic and can be used for any 2D object that can be panned, zoomed or rotated.

The following variables are used for computation and form the [affine coordinate system](https://encyclopediaofmath.org/wiki/Affine_coordinate_system) of the rendered location on the Map. This can be seen as an **orthogonal basis of the reference frame in the Map**. The variables are typed as `Vector2` and can be defined in the context of rendering as following:

 - $\vec O$ or `Origin`: coordinates in Map of the point on the top left of the rendered area
 - $\vec x$ or `HorizontalBasis`: vector in map coordinates to go from one rendered pixel to the neighbor pixel to the right
 - $\vec y$ or `VerticalBasis`: vector in map coordinates to go from one rendered pixel to the neighbor pixel down

The following variables are maintained as help for rendering:

- `Resolution` : size in pixels of the rendered area (`Vector2`)
- `Center` : coordinates in Map of the center of the rendered area (`Vector2`)
- `Diameter` : diameter of the central disk of the rendered area, guaranteed to be visible after any resize or switch landscape/portrait (`float`)
- `Zoom` : zoom factor, i.e. number of pixels per Map distance unit (`float`)
- `Angle` : rotation angle of the basis vectors in Radians (`float`)

### Basic change of reference frame

In the rendering code, it is necessary to perform transformations between the "screen coordinates" (in pixels with $(0,0)$ being the top left corner of the rendered area) and "Map coordinates":

%%svg
<svg><ellipse style="fill:#00ff00;stroke:#000000;" cx="50" cy="50" rx="48" ry="48"><ellipse></svg>